In [11]:
import os
import cv2

def extract_frames(video_path, output_folder, frame_rate=5):
    """
    Extract frames from video at a specified frame rate and save them as images.
    """
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    count = 0
    frame_id = 0

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        if count % frame_rate == 0:  # Capture every nth frame
            frame_name = os.path.join(output_folder, f"{os.path.basename(video_path).split('.')[0]}_{frame_id}.jpg")
            cv2.imwrite(frame_name, frame)
            frame_id += 1
        count += 1
    cap.release()

# Process all videos in dataset
dataset_path = "../data/raw/"
output_path = "../data/processed/images/"

for category in ["Violence", "NonViolence"]:
    input_folder = os.path.join(dataset_path, category)
    output_folder = os.path.join(output_path, category)
    
    for video in os.listdir(input_folder):
        if video.endswith(".mp4"):
            extract_frames(os.path.join(input_folder, video), output_folder)


[h264 @ 0x30d430b60] mb_type 104 in P slice too large at 98 31
[h264 @ 0x30d430b60] error while decoding MB 98 31


In [12]:
import glob

def create_annotations(image_folder, label_folder, class_id):
    os.makedirs(label_folder, exist_ok=True)
    for image_path in glob.glob(f"{image_folder}/*.jpg"):
        label_path = os.path.join(label_folder, os.path.basename(image_path).replace(".jpg", ".txt"))
        with open(label_path, "w") as f:
            f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")  # Full image as object

# Assign labels
create_annotations("../data/processed/images/Violence", "../data/processed/labels/Violence", class_id=0)
create_annotations("../data/processed/images/NonViolence", "../data/processed/labels/NonViolence", class_id=1)

In [13]:
import shutil
import random

def train_test_split(source_folder, dest_folder, split_ratio=(0.7, 0.2, 0.1)):
    """
    Split images and labels into train, val, and test sets.
    """
    os.makedirs(dest_folder, exist_ok=True)
    for subset in ["train", "val", "test"]:
        os.makedirs(os.path.join(dest_folder, "images", subset), exist_ok=True)
        os.makedirs(os.path.join(dest_folder, "labels", subset), exist_ok=True)

    images = sorted(glob.glob(f"{source_folder}/images/*/*.jpg"))
    labels = sorted(glob.glob(f"{source_folder}/labels/*/*.txt"))
    data = list(zip(images, labels))

    random.shuffle(data)

    train_split, val_split = int(len(data) * split_ratio[0]), int(len(data) * (split_ratio[0] + split_ratio[1]))
    train_data, val_data, test_data = data[:train_split], data[train_split:val_split], data[val_split:]

    for subset, subset_data in zip(["train", "val", "test"], [train_data, val_data, test_data]):
        for img, lbl in subset_data:
            shutil.move(img, os.path.join(dest_folder, "images", subset, os.path.basename(img)))
            shutil.move(lbl, os.path.join(dest_folder, "labels", subset, os.path.basename(lbl)))

# Split dataset
train_test_split("../data/processed/", "../data/splited/")

In [14]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # Load pre-trained model
model.train(data="../config/dataset.yaml", epochs=4, imgsz=640, batch=8)

Ultralytics 8.3.91 🚀 Python-3.12.0 torch-2.5.1 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=../config/dataset.yaml, epochs=4, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, f

train: Scanning /Users/wmwong/Documents/GitHub/ThreatSensor/data/splited/labels/train... 39559 images, 


train: New cache created: /Users/wmwong/Documents/GitHub/ThreatSensor/data/splited/labels/train.cache


val: Scanning /Users/wmwong/Documents/GitHub/ThreatSensor/data/splited/labels/val... 11303 images, 0 ba


val: New cache created: /Users/wmwong/Documents/GitHub/ThreatSensor/data/splited/labels/val.cache
Plotting labels to runs/detect/train8/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train8
Starting training for 4 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/4         0G     0.8308      2.882      1.396         23        640:   2%|▏         | 91/4945


KeyboardInterrupt: 

In [ ]:
metrics = model.val()
print(metrics)

In [ ]:
# results = model("test_frame.jpg")
# results.show()

In [11]:
import torch
torch.save(model.model.state_dict(), "../model/trained_model.pt")